**Install library untuk scraping data**

In [ ]:
!pip install google-play-scraper # https://github.com/JoMingyu/google-play-scraper

     |████████████████████████████████| 61kB 3.7MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.0-cp37-none-any.whl size=24070 sha256=ba5fa192cfc79bfc4c7a9b3b10c5fe380d8183c4673d33931ae0d85e6456baa7
  Stored in directory: /root/.cache/pip/wheels/e9/98/5a/917dd15a6f100d35a26c5f6f0ae99895ae861aff74fafa1b08
Successfully built google-play-scraper


**Import library yang telah kita install bersama dengan beberapa library tambahan**

In [ ]:
import pandas as pd # library untuk mengubah hasil scraping menjadi dataframe dan mengelola dataframe
from tqdm import tqdm # library untuk menampilkan progress bar pada proses scraping

from google_play_scraper import Sort, reviews, app # library untuk scraping data

**Definisikan App ID dan masukkan ke dalam function untuk scraping data**

In [ ]:
# mendefinisikan App ID dan variabel hasil scraping data
app_id = ['com.icon.pln123']
app_reviews = []

# function untuk scraping data berada di dalam perulangan For dengan beberapa kondisi tertentu
for app in tqdm(app_id):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      # scraping data ulasan per pengguna
      rvs, _ = reviews(
        app,
        lang = 'id',
        country = 'id',
        sort = sort_order,
        count= 400 if score == 3 else 400,
        filter_score_with = score
      )
      # menggabungkan hasil scraping ulasan per pengguna menjadi satu hasil dalam bentuk List
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = app
      app_reviews.extend(rvs)

100%|██████████| 1/1 [00:14<00:00, 14.31s/it]


**Hitung jumlah ulasan yang kita peroleh dari hasil scraping**

In [ ]:
len(app_reviews) # Karena ada batasan dari library "google-play-scraper", sehingga kita tidak dapat scraping seluruh hasil ulasan pengguna

4000

In [ ]:
print(type(app_reviews))

<class 'list'>


**Konversi hasil scraping data menjadi dataframe**

In [ ]:
df = pd.DataFrame(app_reviews)
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOFlLtbRYUoRV2yDvouVG1wqoRSGcw6VYntuYqP...,Jonatan Tj,https://play-lh.googleusercontent.com/a-/AOh14...,Program nambah id pelanggan masih belum bisa s...,1,14,5.1.4,2021-05-28 08:10:14,Mohon maaf atas kendalanya Kak Jonatan. Penamb...,2021-05-27 15:39:48,most_relevant,com.icon.pln123
1,gp:AOqpTOFRyZxH5GWHkn9aXds3jBBDwEGYYkmtPl4zvs2...,Feri Syaputra,https://play-lh.googleusercontent.com/a/AATXAJ...,Saya melakukan pengaduan pukul 08.28 wib terka...,1,1,5.1.4,2021-05-29 05:18:18,Mohon maaf atas keluhan Kak Feri yang belum te...,2021-05-29 06:50:03,most_relevant,com.icon.pln123
2,gp:AOqpTOG5rK90KLWikYSd2CX6NFgQe7rsTYUTqsRlmgG...,Handra Fatri,https://play-lh.googleusercontent.com/a-/AOh14...,Ajukan penambahan daya pakai promo 202 ribu ya...,1,0,5.1.1,2021-05-29 07:07:21,Mohon maaf atas keluhan yang dialami Kak Handr...,2021-05-29 07:21:39,most_relevant,com.icon.pln123
3,gp:AOqpTOF11keKPxxNojAYe-B1UZce8OfsDEJHrQP5DkQ...,Aldhy Nasution,https://play-lh.googleusercontent.com/a/AATXAJ...,Kenapa untuk yg mau buat pasang baru lokasi ha...,1,0,5.1.1,2021-05-29 01:48:24,Mohon maaf Kak Aldhy .Utk perihal pasang baru ...,2021-05-29 02:14:37,most_relevant,com.icon.pln123
4,gp:AOqpTOGVn3yMQYXAwDXH3Iugt_ZoBscdqrKLQOCqXI-...,Rijalul Fikri,https://play-lh.googleusercontent.com/a-/AOh14...,"Parah, beli token melalui aplikasi sudah 5 har...",1,3,5.0.60,2021-05-22 09:17:51,Mohon maaf atas ketidaknyamanannya Kak Fikri. ...,2021-05-22 10:15:22,most_relevant,com.icon.pln123
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,gp:AOqpTOHIhxhU7FBRpIxIesm24O2mKEwU0ZH5lnigVaB...,David PLN MATANG,https://play-lh.googleusercontent.com/a-/AOh14...,Mntap,5,0,5.1.1,2021-05-26 07:39:33,"Hi Kak David, Wah seneng deh baca ulasannya. T...",2021-05-26 07:44:39,newest,com.icon.pln123
3996,gp:AOqpTOF6Dtgzir-zgLjZ1O0obVWec0adW5RwabkpXyM...,APOH DUQ,https://play-lh.googleusercontent.com/a-/AOh14...,Di aplikasi yang lama ada info tarif listrikny...,5,0,5.1.1,2021-05-26 07:32:12,"Hi Kak Apoh, untuk tarif dasar listrik Kakak d...",2021-05-26 07:53:01,newest,com.icon.pln123
3997,gp:AOqpTOHJkat5SbHRkNO_LUOvsCWqTlokLnfxjILGZp6...,Faisal Reza,https://play-lh.googleusercontent.com/a-/AOh14...,Good... trus update utk lebih baik dalam hal p...,5,0,None,2021-05-26 07:18:52,"Siap Pak Faisal, maaciiw ya udah puas gunakan ...",2021-05-26 07:30:34,newest,com.icon.pln123
3998,gp:AOqpTOGQhN5F2UsTm5njg2XyfB1a9HS0i-05tsB6VqC...,andi azis zalil,https://play-lh.googleusercontent.com/a-/AOh14...,Dukung kok,5,0,5.0.56,2021-05-26 07:13:56,"Halo Kakak Andi, Makasih yaa buat review & rat...",2021-05-26 07:29:10,newest,com.icon.pln123


In [ ]:
df['score'].value_counts()

5    800
3    800
1    800
4    800
2    800
Name: score, dtype: int64

**Hapus ulasan dengan rating 3 bintang agar dataset balanced**

In [ ]:
df.drop(df.loc[df['score']==3].index, inplace=True)
len(df)

3200

**Hapus kolom yang tidak digunakan**

In [ ]:
df.drop(['reviewId', 'userImage', 'userName', 'score', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt','sortOrder','appId'], axis=1, inplace=True)

In [ ]:
pd.options.display.max_colwidth=1000
df.head()

,content
0,"Program nambah id pelanggan masih belum bisa saya input, ket: selalu saja, sistem sedang sibuk,silakan coba beberapa saat lagi V001. Kenapa keterangan tiket nomer tiket 1183886 sudah terselesaikan/ditutup, sedangkan masalah masih belum terselesaikan (semua instruksi dari balasan email sdh saya dicoba). Ini balasan email dari help desk Kepada Yth. Bapak/Ibu, Keluhan Bapak/Ibu dengan Nomor Tiket 1183886 terkait NEW PLN MOBILE KELUHAN telah di selesaikan"
1,Saya melakukan pengaduan pukul 08.28 wib terkait matinya lampu dirumah saya sedangkan dirumah tetangga tidak. Riwayat status di aplikasi penugasan regu pukul 08.31 wib dan dalam perjalanan pukul 10.52 wib. Sampai sekarang pukul 12.18 petugas pln belum juga datang.
2,Ajukan penambahan daya pakai promo 202 ribu yang lewat aplikasi ini. Udah 2 minggu gak ada penanganan. Bikin pengaduan statusnya langsung selesai tanpa tindak lanjut. Ini lah potret kalau BUMN masih diisi oleh orang-oranh Old Mind dan Korup. Gak bakalan maju Negara kita ini. Warung kecil jualan token listrik bisa untung. PLN jualan listrik tp Utangnya nambah. Hancur hancur.
3,"Kenapa untuk yg mau buat pasang baru lokasi hanya di jawa timur? Tolong dong penjelasannya, saya download apps ini karena mau pasang baru, dan saya sudah coba dr website, tp gk bs makanya saya download apps ini eh malah makin gk bs tolong dong diperbaiki apps ini, terima kasih"
4,"Parah, beli token melalui aplikasi sudah 5 hari keterangannya masih diproses. Sudah melakukan PENGADUAN melalui aplikasi juga, tidak ada tindak lanjut tapi laporan saya keterangannya sudah selesai. Sudah tiga kali pelaporan dan tidak ada kemajuan sama sekali."


**Konversi dataframe menjadi bentuk CSV**

In [ ]:
df.to_csv("raw_dataset.csv", index=None, header=None)

**Download file CSV**

In [ ]:
from google.colab import files
files.download("raw_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>